# Objective

The objective of this analysis is to enhance our knowledge of the signaling network of cyanothece. We believe that the signaling network in cyanothece involves the circadian clock genes and two-component sensors and regulators. In cyanobacteria signaling network, two component systems play a major role and connect the circadian clock genes with downstream genes. The primary component is a kinase and secondary component is a regulator. Since the kinase and the regulator are part of the same two component unit, their expression profiles must be very similar and we intend to quantify that correlation using Mutual Information. If the mutual information between the expression profile of a kinase and a regulator is high, it may be possible that they are a part of the same two component system. Similarly, if a component of the two-component system interacts with the clock genes, their mutual information must also be high. Thus, using mutual information as a metric we aim to gain more insight into the signaling network of cyanothece.

# Steps

1. We'll use the microarray expression dataset obtained by Stockel et al; link: https://europepmc.org/article/pmc/pmc2329701
2. We'll select the genes which are part of any two-component system and the circadian clock genes using the genomic annotations of Cyanothece obtained from the studies of Welsh et al; link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2567498/
3. We'll use scikit-learn's mutual information regression to calculate the correlation between the primary and secondary components and also between the clock genes.
4. Using a list of probable sensor-regulator pairs sorted by the mutual information value between them, we can predict which sensor is mostly likely to interact with a regulator and vice versa and also with the clock. 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy as sp
import numpy as np
import multiprocessing as mp
from sklearn.feature_selection import mutual_info_regression

# Data Preprocessing

In [3]:
df_stockel = pd.read_csv('MicroarrayData/StockelProcessed.csv')

In [4]:
df_stockel.head()

,Contig,ORF,Day1_D1,Day1_D5,Day1_D9,Day1_L1,Day1_L5,Day1_L9,Day2_D1,Day2_D5,Day2_D9,Day2_L1,Day2_L5,Day2_L9
0,Contig0.10_10_14370_15446,cce_4187,-0.74,-0.62,-0.13,0.35,0.26,-0.16,-0.87,-0.43,-0.12,0.38,0.37,-0.04
1,Contig0.10_11_16336_15422,cce_4186,-0.33,-0.64,-0.31,0.16,0.41,0.23,-0.42,-0.45,-0.30,0.06,0.34,0.23
2,Contig0.10_12_16503_17552,cce_4185,-0.46,-0.41,-0.41,0.66,-0.20,-0.04,-0.46,-0.28,-0.46,0.48,-0.01,0.04
3,Contig0.10_13_17679_17981,cce_4184,-0.23,0.27,0.13,0.08,-0.12,-0.08,-0.23,0.11,0.12,0.18,-0.05,-0.08
4,Contig0.10_14_18000_18698,cce_4183,-0.18,0.18,-0.02,-0.09,-0.22,-0.15,-0.21,0.09,-0.04,-0.08,-0.13,-0.16


A quick look at the above dataframe will show that there are repeats in the ORF column

In [5]:
len(df_stockel.ORF)==len(set(df_stockel.ORF))

False

We need to find the ORFs that are duplicated.

In [6]:
duplicates = df_stockel.loc[df_stockel.duplicated(subset='ORF',keep=False)].sort_values(by=['ORF'])

In [7]:
duplicates.head()

,Contig,ORF,Day1_D1,Day1_D5,Day1_D9,Day1_L1,Day1_L5,Day1_L9,Day2_D1,Day2_D5,Day2_D9,Day2_L1,Day2_L5,Day2_L9
4089,Contig62.1_1_1274_207,cce_0015,0.09,0.12,0.09,0.15,0.02,-0.01,0.06,0.23,0.20,0.12,0.03,0.01
3925,Contig6.1_2_481_873,cce_0015,-0.12,0.10,0.05,0.14,-0.11,-0.09,-0.09,0.09,0.02,0.11,-0.02,-0.07
4098,Contig62.2_1_424_44,cce_0027,0.11,0.03,0.13,-0.04,0.09,0.04,0.10,0.04,0.07,-0.07,-0.04,-0.02
4088,Contig62.1_14_11081_10533,cce_0027,-0.03,NaN,-0.03,-0.06,-0.08,-0.07,-0.08,-0.05,-0.07,-0.08,-0.12,-0.08
4698,Contig80.1_14_16018_15353,cce_0067,-0.10,0.11,-0.14,-0.21,-0.23,-0.21,-0.05,-0.08,-0.10,-0.22,-0.26,-0.18


It seems that some of them has NaN values, we will take care of them later. Firstly, we will take the mean of the expression values of the duplicates.

In [8]:
mean_columns = list(duplicates.columns)
mean_columns.remove('Contig')
mean_columns.remove('ORF')

In [9]:
df_expression = df_stockel.groupby('ORF')[mean_columns].mean().reset_index()

Now we download the Cyanothece Database with genomic annotations. 

In [10]:
GenCyanoDB = pd.read_excel('GenCyanoDB.xlsx',index_col=0,usecols=[0,1,2,3])

In [11]:
GenCyanoDB.head()

,ORF,Function,CommonName
0,cce_0001,hypothetical protein,cce_0001
1,cce_0002,alcohol dehydrogenase,cce_0002
2,cce_0003,hypothetical protein,cce_0003
3,cce_0004,cation efflux system membrane protein,czcA
4,cce_0005,conserved hypothetical protein,cce_0005


First, let's find how many genes have the keyword regulator, two-component, kinase, sensor or circadian in their functional annotation. 

In [12]:
twoComponents = GenCyanoDB[GenCyanoDB['Function'].str.contains("two-component|kinase|regulator|sensor|circadian")]

In [13]:
len(twoComponents),len(twoComponents.loc[twoComponents.ORF.isin(df_stockel.ORF)])

(211, 204)

It's clear from above that we do not have expression profiles for 7 genes. Let's find which ones.

In [14]:
twoComponents.loc[~twoComponents.ORF.isin(df_stockel.ORF)]

,ORF,Function,CommonName
11,cce_0012,two-component response regulator,cce_0012
798,cce_0800,acetate kinase,ackA1
4029,cce_4034,adenylate kinase,adk
4070,cce_4075,putative ATP-NAD/AcoX kinase,cce_4075
4586,cce_4591,guanylate kinase,gmk
4654,cce_4659,two-component response regulator,cce_4659
4710,cce_4715,putative circadian clock protein,kaiB2


We cannot include these in our analysis. It's a shame that we cannot include KaiB2.

Next we need to merge the 2 dataframes.

In [15]:
twoComponentsExp = df_expression.merge(twoComponents,on='ORF',how='inner')

First, we will check the length of the dataframe.

In [16]:
assert len(twoComponentsExp)==len(twoComponents.loc[twoComponents.ORF.isin(df_stockel.ORF)])

In [17]:
twoComponentsExp.head()

,ORF,Day1_D1,Day1_D5,Day1_D9,Day1_L1,Day1_L5,Day1_L9,Day2_D1,Day2_D5,Day2_D9,Day2_L1,Day2_L5,Day2_L9,Function,CommonName
0,cce_0016,0.14,0.05,0.06,0.02,0.06,0.07,0.20,0.10,0.14,0.04,0.05,0.09,two-component sensor histidine kinase,cce_0016
1,cce_0115,-0.43,-0.18,-0.20,0.04,NaN,-0.12,-0.43,-0.10,-0.20,-0.07,-0.05,-0.09,response regulator,cce_0115
2,cce_0123,0.18,0.51,0.44,-0.27,-0.33,-0.40,0.07,0.38,0.41,-0.19,-0.18,-0.38,thiamine monophosphate kinase,thiL
3,cce_0145,0.07,-0.08,0.22,-0.03,0.54,0.16,0.06,0.06,0.22,-0.06,0.20,0.09,putative circadian clock protein,kaiB4
4,cce_0164,-0.15,-0.24,0.03,0.30,0.24,0.16,-0.22,-0.06,0.15,0.30,0.19,0.19,two-component sensor histidine kinase,cce_0164


# Mutual Information Calculation

Next we need to develop a ranked list of the interactions between the genes of the twoCompenentsExp dataframe with each other. We hope to find a primary component with a high correlation to a secondary component and vice versa. That will help us to conclude that they are a part of the two-component regulation system. As a correlation metric, we will use mutual information since it can capture non-linear interactions. 

In [18]:
class Interaction:
    def __init__(self,Exp_data,gene='all',mi_thresh=0):
        self.Exp_data = Exp_data
        if self.Exp_data.isnull().values.any():
            self.Exp_df = self.Exp_data.iloc[:,:-2].set_index('ORF').interpolate(method='linear',axis=1,limit_direction='both').T
        else:
            self.Exp_df = self.Exp_data.iloc[:,:-2].set_index('ORF').T
        if gene=='all':
            self.mi_dict = self._get_dict()
        else:
            self.gene_orf = gene
            self.mi_list = self._miscorelist(self.gene_orf)
            self.mi_thresh = mi_thresh
            self.df = self._get_df(self.mi_list,self.mi_thresh)
           
    
    def _get_dict(self):
        all_genes = list(self.Exp_df.columns)
        pool = mp.Pool(mp.cpu_count())
        results = pool.map(self._miscorelist,all_genes)
        fast_dict= dict(zip(all_genes,results))
        return fast_dict

    
    def _miscorelist(self,gene):
        all_other_genes_df = self.Exp_df.loc[:,self.Exp_df.columns!=gene]
        all_other_genes = np.array(all_other_genes_df.columns)
        this_gene_df = self.Exp_df[gene]
        mi_score = mutual_info_regression(all_other_genes_df,this_gene_df,discrete_features=False,random_state=7)
        miscore_genes = list(zip(all_other_genes,mi_score))
        sorted_miscore = sorted(miscore_genes,key = lambda x:x[1],reverse=True)
        return sorted_miscore
    
    def _get_df(self,mi_list,mi_thresh):
        my_dict = {'orf':[],'function':[],'CommonName':[],'mi':[]}
        for orf,mi in mi_list:
            if mi<=mi_thresh:
                break

            my_dict['orf'].append(orf)
            my_dict['function'].append(twoComponentsExp.loc[twoComponentsExp.ORF==orf].Function.values[0])
            my_dict['CommonName'].append(twoComponentsExp.loc[twoComponentsExp.ORF==orf].CommonName.values[0])
            my_dict['mi'].append(mi)

        return pd.DataFrame(my_dict)
    
    def get_twoComponentHybrids(self):
        return self.df.loc[self.df.function.str.contains('two-component') & self.df.function.str.contains('hybrid')]
    
    def get_twoComponentSensors(self):
        return self.df.loc[self.df.function.str.contains('two-component') & self.df.function.str.contains('sensor') & ~self.df.function.str.contains('hybrid')]
    
    def get_twoComponentRegulators(self):
        return self.df.loc[self.df.function.str.contains('two-component') & self.df.function.str.contains('regulator') & ~self.df.function.str.contains('hybrid')]

    def get_other_clock(self):
        return self.df.loc[self.df.function.str.contains('clock protein')]

In [19]:
mi = Interaction(twoComponentsExp)

# Cyanothece clock genes interaction

In [20]:
clock_genes = {i:GenCyanoDB.loc[GenCyanoDB.CommonName==i].ORF.values[0] for i in ['kaiA','kaiB1','kaiB3',
                                                                                  'kaiB4','kaiC1','kaiC2']}

## KaiA

In [21]:
kA = Interaction(twoComponentsExp,clock_genes['kaiA'],mi_thresh=0)

In [22]:
kA.get_other_clock()

,orf,function,CommonName,mi
18,cce_0423,circadian clock protein,kaiB1,0.624044
56,cce_0422,circadian clock protein,kaiC1,0.357973


## KaiB1

In [23]:
kB1 = Interaction(twoComponentsExp,clock_genes['kaiB1'],mi_thresh=0)

In [24]:
kB1.get_other_clock()

,orf,function,CommonName,mi
11,cce_0424,circadian clock protein,kaiA,0.624044
145,cce_4716,circadian clock protein,kaiC2,0.048746
150,cce_0422,circadian clock protein,kaiC1,0.027814


## KaiB3

In [25]:
kB3 = Interaction(twoComponentsExp,clock_genes['kaiB3'],mi_thresh=0)

In [26]:
kB3.get_other_clock()

,orf,function,CommonName,mi
128,cce_0422,circadian clock protein,kaiC1,0.010453


## KaiB4

In [27]:
kB4 = Interaction(twoComponentsExp,clock_genes['kaiB4'],mi_thresh=0)

In [28]:
kB4.get_other_clock()

,orf,function,CommonName,mi
0,cce_4716,circadian clock protein,kaiC2,0.392463
32,cce_0422,circadian clock protein,kaiC1,0.136742


## KaiC1

In [29]:
kC1 = Interaction(twoComponentsExp,clock_genes['kaiC1'],mi_thresh=0)

In [30]:
kC1.get_other_clock()

,orf,function,CommonName,mi
23,cce_0424,circadian clock protein,kaiA,0.427417
95,cce_0145,putative circadian clock protein,kaiB4,0.148647
133,cce_0423,circadian clock protein,kaiB1,0.039719
135,cce_4716,circadian clock protein,kaiC2,0.034957
144,cce_0435,circadian clock protein,kaiB3,0.010453


## KaiC2

In [31]:
kC2 = Interaction(twoComponentsExp,clock_genes['kaiC2'],mi_thresh=0)

In [32]:
kC2.get_other_clock()

,orf,function,CommonName,mi
12,cce_0145,putative circadian clock protein,kaiB4,0.406352
116,cce_0423,circadian clock protein,kaiB1,0.036842
123,cce_0422,circadian clock protein,kaiC1,0.023052


# Finding the sensors and regulators that interact with any clock gene.

## Sensors

In [32]:
kA_sensors = kA.get_twoComponentSensors().orf.values
kB1_sensors = kB1.get_twoComponentSensors().orf.values
kB3_sensors = kB3.get_twoComponentSensors().orf.values
kB4_sensors = kB4.get_twoComponentSensors().orf.values
kC1_sensors = kC1.get_twoComponentSensors().orf.values
kC2_sensors = kC2.get_twoComponentSensors().orf.values

In [33]:
all_sensors = np.concatenate((kA_sensors,kB1_sensors,kB3_sensors,kB4_sensors,kC1_sensors,kC2_sensors))

In [34]:
sensor_set = set(all_sensors)
twoComponentsExp.loc[twoComponentsExp.ORF.isin(sensor_set)]

,ORF,Day1_D1,Day1_D5,Day1_D9,Day1_L1,Day1_L5,Day1_L9,Day2_D1,Day2_D5,Day2_D9,Day2_L1,Day2_L5,Day2_L9,Function,CommonName
0,cce_0016,0.14,0.05,0.06,0.02,0.06,0.07,0.20,0.10,0.14,0.04,0.05,0.09,two-component sensor histidine kinase,cce_0016
4,cce_0164,-0.15,-0.24,0.03,0.30,0.24,0.16,-0.22,-0.06,0.15,0.30,0.19,0.19,two-component sensor histidine kinase,cce_0164
7,cce_0220,0.67,0.24,-0.12,-0.53,0.25,0.13,0.58,0.17,-0.04,-0.37,-0.16,-0.13,two-component sensor histidine kinase,cce_0220
8,cce_0257,0.01,0.36,0.11,-0.09,-0.06,0.06,0.13,0.12,0.06,-0.04,-0.01,-0.03,two-component sensor histidine kinase,cce_0257
10,cce_0297,-0.16,-0.08,0.14,0.07,0.04,0.19,-0.12,-0.02,0.15,0.05,0.07,0.11,two-component sensor histidine kinase,cce_0297
41,cce_0888,0.29,0.40,0.07,-0.10,-0.09,0.02,0.26,0.42,0.29,0.08,0.03,0.04,two-component sensor histidine kinase,nblS
44,cce_0969,1.08,0.29,-0.18,-0.65,-0.43,-0.49,1.30,0.17,-0.08,-0.45,-0.40,-0.40,two-component sensor histidine kinase,cce_0969
56,cce_1280,-0.19,-0.22,-0.25,0.69,-0.03,-0.28,-0.19,-0.15,-0.10,0.41,-0.25,-0.26,two-component sensor histidine kinase,cce_1280
63,cce_1467,0.20,-0.01,0.06,0.27,0.15,0.04,0.19,0.02,0.02,0.07,0.02,-0.03,two-component sensor histidine kinase,cce_1467
65,cce_1519,NaN,0.10,-0.12,-0.01,-0.05,0.06,0.10,0.03,0.15,NaN,0.01,0.11,two-component sensor histidine kinase,cce_1519


## Regulators

In [35]:
kA_regulators = kA.get_twoComponentRegulators().orf.values
kB1_regulators = kB1.get_twoComponentRegulators().orf.values
kB3_regulators = kB3.get_twoComponentRegulators().orf.values
kB4_regulators = kB4.get_twoComponentRegulators().orf.values
kC1_regulators = kC1.get_twoComponentRegulators().orf.values
kC2_regulators = kC2.get_twoComponentRegulators().orf.values

In [36]:
all_regulators = np.concatenate((kA_regulators,kB1_regulators,kB3_regulators,kB4_regulators,kC1_regulators,kC2_regulators))

In [37]:
regulator_set = set(all_regulators)
twoComponentsExp.loc[twoComponentsExp.ORF.isin(regulator_set)]

,ORF,Day1_D1,Day1_D5,Day1_D9,Day1_L1,Day1_L5,Day1_L9,Day2_D1,Day2_D5,Day2_D9,Day2_L1,Day2_L5,Day2_L9,Function,CommonName
5,cce_0165,0.060000,0.010,-0.040000,0.120000,0.020000,-0.010,0.010000,0.04,0.010,0.120,0.020000,0.010,two-component response regulator,cce_0165
9,cce_0289,-0.220000,-0.040,0.140000,0.340000,-0.030000,-0.190,-0.330000,0.12,0.140,0.230,-0.110000,-0.250,two-component response regulator,cce_0289
11,cce_0298,-2.500000,-2.330,-0.700000,-0.590000,0.510000,1.060,-2.090000,-2.28,-0.690,-0.630,0.570000,1.100,two-component response regulator,rpaA
17,cce_0446,-0.350000,-0.480,-0.650000,1.230000,0.150000,-0.780,-0.550000,-0.43,-0.510,0.920,-0.640000,-0.710,two-component response regulator,cce_0446
26,cce_0657,NaN,0.150,0.110000,0.160000,-0.040000,-0.030,-0.040000,0.18,0.230,0.150,NaN,-0.020,two-component response regulator,cce_0657
28,cce_0678,-0.450000,-0.230,0.050000,0.560000,0.270000,-0.150,-0.610000,0.05,0.050,0.410,0.140000,-0.070,two-component response regulator,cce_0678
32,cce_0712,0.310000,0.390,0.250000,0.160000,-0.010000,-0.010,0.250000,0.15,0.250,0.130,0.060000,0.020,two-component response regulator,cce_0712
33,cce_0713,0.290000,0.020,0.020000,0.500000,0.230000,-0.120,0.220000,0.04,-0.010,0.260,-0.040000,-0.070,two-component response regulator,cce_0713
37,cce_0754,-0.400000,-0.635,-0.490000,1.125000,0.170000,-0.540,-0.540000,-0.40,-0.420,0.800,-0.410000,-0.500,two-component response regulator,cce_0754
45,cce_0970,1.410000,0.570,-0.060000,-1.180000,-0.730000,-0.740,1.490000,0.23,-0.290,-1.010,-0.580000,-0.700,two-component transcription regulator,cce_0970


# Finding the most common sensors and regulators that interact with all the clock genes

## Sensors

In [38]:
sensors = [kA_sensors,kB1_sensors,kB3_sensors,kB4_sensors,kC1_sensors,kC2_sensors]

def most_common_elements(given_set):
    main_set = set(given_set[0])

    for sarray in given_set[1:]:
        sset = set(sarray)
        main_set.intersection_update(sset)
    return main_set

In [39]:
twoComponentsExp.loc[twoComponentsExp.ORF.isin(most_common_elements(sensors))]

,ORF,Day1_D1,Day1_D5,Day1_D9,Day1_L1,Day1_L5,Day1_L9,Day2_D1,Day2_D5,Day2_D9,Day2_L1,Day2_L5,Day2_L9,Function,CommonName
4,cce_0164,-0.15,-0.24,0.03,0.30,0.24,0.16,-0.22,-0.06,0.15,0.3,0.19,0.19,two-component sensor histidine kinase,cce_0164
138,cce_3379,-0.28,-0.13,-0.18,-0.07,0.05,0.09,-0.28,-0.12,-0.14,-0.1,-0.04,0.08,two-component sensor histidine kinase,cce_3379


## Regulators

In [40]:
regulators = [kA_regulators,kB1_regulators,kB3_regulators,kB4_regulators,kC1_regulators,kC2_regulators]

In [41]:
twoComponentsExp.loc[twoComponentsExp.ORF.isin(most_common_elements(regulators))]

,ORF,Day1_D1,Day1_D5,Day1_D9,Day1_L1,Day1_L5,Day1_L9,Day2_D1,Day2_D5,Day2_D9,Day2_L1,Day2_L5,Day2_L9,Function,CommonName
45,cce_0970,1.41,0.57,-0.06,-1.18,-0.73,-0.74,1.49,0.23,-0.29,-1.01,-0.58,-0.70,two-component transcription regulator,cce_0970
66,cce_1520,0.61,0.38,-0.04,-0.30,-0.24,-0.02,0.51,0.21,0.31,-0.26,-0.11,-0.01,two-component response regulator,cce_1520
107,cce_2365,-0.02,-0.10,0.10,0.21,0.23,0.12,-0.08,0.04,0.10,0.18,0.19,0.15,two-component response regulator,cce_2365
146,cce_3559,0.18,0.11,0.10,0.02,0.06,0.12,0.17,0.12,0.10,0.04,0.07,0.06,two-component response regulator,cce_3559


# Finding the kaiA,kaiB,kaiC combination that interacts with the maximum number of sensors and regulators

In [42]:
from itertools import product
clock_gene_copies = list(clock_genes.keys())
kaiA_copies = clock_gene_copies[0:1]
kaiB_copies = clock_gene_copies[1:4]
kaiC_copies = clock_gene_copies[4:]
print(list(product(kaiA_copies,kaiB_copies,kaiC_copies)))

[('kaiA', 'kaiB1', 'kaiC1'), ('kaiA', 'kaiB1', 'kaiC2'), ('kaiA', 'kaiB3', 'kaiC1'), ('kaiA', 'kaiB3', 'kaiC2'), ('kaiA', 'kaiB4', 'kaiC1'), ('kaiA', 'kaiB4', 'kaiC2')]


In [43]:
clock_gene_sensors = dict(zip(clock_gene_copies,sensors))
clock_gene_regulators = dict(zip(clock_gene_copies,regulators))

number_dict = {}

for a,b,c in product(kaiA_copies,kaiB_copies,kaiC_copies):
    sensors_list = [clock_gene_sensors[i] for i in (a,b,c)]
    regulators_list = [clock_gene_regulators[i] for i in (a,b,c)]
    sensor_len = len(twoComponentsExp.loc[twoComponentsExp.ORF.isin(most_common_elements(sensors_list))])
    regulator_len = len(twoComponentsExp.loc[twoComponentsExp.ORF.isin(most_common_elements(regulators_list))])
    total_len = sensor_len+regulator_len
    number_dict[(a,b,c)] = [sensor_len,regulator_len,total_len]
    
{k:v for k,v in sorted(number_dict.items(),key=lambda x: x[1][2],reverse=True)}

{('kaiA', 'kaiB1', 'kaiC1'): [13, 19, 32],
 ('kaiA', 'kaiB1', 'kaiC2'): [13, 19, 32],
 ('kaiA', 'kaiB3', 'kaiC2'): [10, 13, 23],
 ('kaiA', 'kaiB3', 'kaiC1'): [9, 12, 21],
 ('kaiA', 'kaiB4', 'kaiC2'): [6, 12, 18],
 ('kaiA', 'kaiB4', 'kaiC1'): [4, 12, 16]}

# cce_0678 case study

In [44]:
cce_0678 = Interaction(twoComponentsExp,'cce_0678',mi_thresh=0)
cce_0678.get_twoComponentSensors()

,orf,function,CommonName,mi
12,cce_1983,"probable phytochrome A, two-component sensor p...",aphA,0.507973
30,cce_0220,two-component sensor histidine kinase,cce_0220,0.371861
33,cce_1535,two-component sensor histidine kinase,cce_1535,0.358733
39,cce_0969,two-component sensor histidine kinase,cce_0969,0.339719
42,cce_3894,two-component sensor histidine kinase,cce_3894,0.332179
55,cce_1878,two-component sensor histidine kinase,cce_1878,0.272953
59,cce_0297,two-component sensor histidine kinase,cce_0297,0.264024
65,cce_0016,two-component sensor histidine kinase,cce_0016,0.252913
66,cce_0164,two-component sensor histidine kinase,cce_0164,0.250234
81,cce_4006,two-component sensor histidine kinase,cce_4006,0.198019


# rpaA case study

In [45]:
cce_0298 = Interaction(twoComponentsExp,'cce_0298',mi_thresh=0)
cce_0298.get_twoComponentSensors()

,orf,function,CommonName,mi
9,cce_0164,two-component sensor histidine kinase,cce_0164,0.690711
14,cce_1983,"probable phytochrome A, two-component sensor p...",aphA,0.617000
27,cce_0888,two-component sensor histidine kinase,nblS,0.526822
37,cce_3379,two-component sensor histidine kinase,cce_3379,0.504401
44,cce_4426,two-component sensor histidine kinase,cce_4426,0.477781
49,cce_0297,two-component sensor histidine kinase,cce_0297,0.461346
57,cce_4097,two-component sensor serine/threonine kinase,cce_4097,0.419877
68,cce_2546,probable two-component sensor histidine kinase,cce_2546,0.372027
86,cce_0969,two-component sensor histidine kinase,cce_0969,0.297060
95,cce_2366,two-component sensor histidine kinase,cce_2366,0.242496


# sasA case study

In [46]:
cce_1751 = Interaction(twoComponentsExp,'cce_1751',mi_thresh=0)
cce_1751.get_twoComponentRegulators()

,orf,function,CommonName,mi
2,cce_4002,two-component response regulator,rpaB,0.655195
6,cce_1808,probable two-component system response regulat...,cce_1808,0.583667
17,cce_4714,"two-component response regulator, NarL subfamily",cce_4714,0.445241
18,cce_3895,two-component response regulator,cce_3895,0.439917
20,cce_0289,two-component response regulator,cce_0289,0.409361
22,cce_0678,two-component response regulator,cce_0678,0.387834
24,cce_1952,two-component response regulator receiver protein,cce_1952,0.372655
25,cce_3960,two-component response regulator,cce_3960,0.359924
26,cce_4416,two-component response regulator,cce_4416,0.355195
27,cce_3557,two-component response regulator,cce_3557,0.348250


# rpaB case study

In [69]:
cce_4002 = Interaction(twoComponentsExp,'cce_4002',mi_thresh=0)
cce_4002.df.head(10)

,orf,function,CommonName,mi
0,cce_0669,6-phosphofructokinase,pfkA1,0.697655
1,cce_1751,adaptive-response sensory histidine kinase,sasA,0.682973
2,cce_0435,circadian clock protein,kaiB3,0.673250
3,cce_0298,two-component response regulator,rpaA,0.668654
4,cce_3420,pyruvate kinase,pykF1,0.640512
5,cce_1983,"probable phytochrome A, two-component sensor p...",aphA,0.628806
6,cce_0453,putative transcriptional regulator AbrB,cce_0453,0.616074
7,cce_3895,two-component response regulator,cce_3895,0.592000
8,cce_2971,putative polyphosphate kinase 2,cce_2971,0.570869
9,cce_4183,two-component response regulator,cce_4183,0.566570


# Conclusions

1. cce_1983/aphA might be a photoreceptor that regulates the clock genes and the other TFs. 
2. cce_0888/nblS is another interesting component that interacts with the clock genes. In 7942, it is shown to be a gene involved in photosynthesis related gene expression during high light and nutrient stress. 
3. In the previous literature review study report, cce_0678 was proposed to interact with the RubisCo genes. In this study, it was shown that there is a very high mutual information score between cce_0678 and the probable photoreceptor aphA discussed above. This further highlights it's importance as a regulator. 
4. rpaA and rpaB are equally important in cyanothece signaling network according to this analysis. 
5. sasA may not be the kinase that interacts with rpaA in cyanothece. On the otherhand, rpaB may be the regulator that interacts with sasA. 
6. The KaiB copies may be present not just to maintain robustness but they may play important roles in the signaling network. 